In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
# Input data files are available in the "../data/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../data"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
# load data
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

# remove constant columns
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

# remove duplicated columns
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,df_train[c[j]].values):
            remove.append(c[j])

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

y_train = df_train['TARGET'].values
X_train = df_train.drop(['ID','TARGET'], axis=1).values

id_test = df_test['ID']
X_test = df_test.drop(['ID'], axis=1).values

# length of dataset
len_train = len(X_train)
len_test  = len(X_test)

test.csv
train.csv



In [5]:
for col in df_train.columns:
    print col

ID
var3
var15
imp_ent_var16_ult1
imp_op_var39_comer_ult1
imp_op_var39_comer_ult3
imp_op_var40_comer_ult1
imp_op_var40_comer_ult3
imp_op_var40_efect_ult1
imp_op_var40_efect_ult3
imp_op_var40_ult1
imp_op_var41_comer_ult1
imp_op_var41_comer_ult3
imp_op_var41_efect_ult1
imp_op_var41_efect_ult3
imp_op_var41_ult1
imp_op_var39_efect_ult1
imp_op_var39_efect_ult3
imp_op_var39_ult1
imp_sal_var16_ult1
ind_var1_0
ind_var1
ind_var5_0
ind_var5
ind_var6_0
ind_var6
ind_var8_0
ind_var8
ind_var12_0
ind_var12
ind_var13_0
ind_var13_corto_0
ind_var13_corto
ind_var13_largo_0
ind_var13_largo
ind_var13_medio_0
ind_var13
ind_var14_0
ind_var14
ind_var17_0
ind_var17
ind_var18_0
ind_var19
ind_var20_0
ind_var20
ind_var24_0
ind_var24
ind_var25_cte
ind_var26_0
ind_var26_cte
ind_var25_0
ind_var30_0
ind_var30
ind_var31_0
ind_var31
ind_var32_cte
ind_var32_0
ind_var33_0
ind_var33
ind_var34_0
ind_var37_cte
ind_var37_0
ind_var39_0
ind_var40_0
ind_var40
ind_var41_0
ind_var44_0
ind_var44
num_var1_0
num_var1
num_var4
num_var

In [8]:
rf = RandomForestClassifier(n_estimators=500)
rf.fit(X_train, y_train)
print "Features sorted by their score:"


Features sorted by their score:


In [10]:
var_imp = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), df_train.columns[1:]), 
             reverse=True)

In [13]:
imp_vars = []
non_imp_vars = 0
for varimp,var in var_imp:
    if varimp>0:
        imp_vars.append(var)


In [14]:
imp_vars

['var38',
 'var15',
 'saldo_medio_var5_ult3',
 'saldo_medio_var5_hace3',
 'num_var45_ult3',
 'saldo_var30',
 'saldo_medio_var5_hace2',
 'num_var45_hace3',
 'saldo_var42',
 'num_var45_hace2',
 'num_var22_ult3',
 'saldo_medio_var5_ult1',
 'saldo_var5',
 'num_var45_ult1',
 'num_var22_hace3',
 'num_var22_hace2',
 'num_med_var45_ult3',
 'var36',
 'num_var22_ult1',
 'num_meses_var39_vig_ult3',
 'num_meses_var5_ult3',
 'num_med_var22_ult3',
 'imp_op_var41_ult1',
 'imp_op_var39_ult1',
 'imp_op_var41_comer_ult3',
 'imp_op_var39_comer_ult3',
 'num_var35',
 'imp_trans_var37_ult1',
 'num_var4',
 'imp_op_var41_efect_ult3',
 'imp_op_var39_efect_ult3',
 'imp_op_var41_comer_ult1',
 'imp_op_var39_comer_ult1',
 'var3',
 'saldo_var37',
 'num_op_var41_ult3',
 'num_op_var39_ult3',
 'imp_op_var39_efect_ult1',
 'imp_op_var41_efect_ult1',
 'imp_ent_var16_ult1',
 'num_op_var41_comer_ult3',
 'num_op_var39_comer_ult3',
 'num_op_var41_ult1',
 'num_op_var39_hace2',
 'num_var30',
 'num_op_var41_hace2',
 'num_op_var

In [17]:
df_train=df_train[imp_vars]

In [18]:
correlations =  df_train.corr()

In [48]:
for i in range(0,len(imp_vars)-1):
    for j in range(i+1,len(imp_vars)):
        if correlations[imp_vars[i]][imp_vars[j]]>0.99:
            print imp_vars[i],"and",imp_vars[j],"are correlated with correlation:" , correlations[imp_vars[i]][imp_vars[j]]

num_var45_ult3 and num_med_var45_ult3 are correlated with correlation: 0.997806009995
imp_op_var41_ult1 and imp_op_var39_ult1 are correlated with correlation: 0.99110268492
imp_op_var41_efect_ult3 and imp_op_var39_efect_ult3 are correlated with correlation: 0.999270934254
imp_op_var39_efect_ult1 and imp_op_var41_efect_ult1 are correlated with correlation: 0.998378134203
num_op_var39_hace2 and num_op_var41_hace2 are correlated with correlation: 0.99259644747
num_op_var41_efect_ult3 and num_op_var39_efect_ult3 are correlated with correlation: 0.999244416638
num_op_var41_efect_ult1 and num_op_var39_efect_ult1 are correlated with correlation: 0.99896594905
num_var5 and ind_var5 are correlated with correlation: 0.99370918937
num_var8_0 and ind_var8_0 are correlated with correlation: 0.999793191936
saldo_var12 and saldo_var24 are correlated with correlation: 0.996932397338
num_var24_0 and ind_var24_0 are correlated with correlation: 0.997448385603
num_var40_0 and num_var1_0 are correlated wi

#pca

In [35]:
from sklearn import preprocessing
from sklearn.decomposition import PCA

pca = PCA(n_components=.99)

In [36]:
from sklearn import preprocessing
X_normalized = preprocessing.normalize(X_train, norm='l2')

In [37]:
x_pca = pca.fit_transform(X_normalized)

In [38]:
pca.explained_variance_

array([  5.91985703e-02,   1.87563839e-02,   8.12114346e-03,
         4.25923015e-03,   3.57316583e-03,   2.22360742e-03,
         1.45692967e-03,   1.34116785e-03,   1.12840309e-03,
         6.37991060e-04,   5.86282684e-04,   5.14901936e-04,
         4.46508613e-04,   4.24916968e-04,   4.17206222e-04,
         3.04049729e-04,   2.80740025e-04,   2.46933522e-04,
         2.26709042e-04,   2.14752935e-04,   1.95583816e-04,
         1.46629586e-04,   1.40411435e-04,   1.25520450e-04,
         1.22196583e-04,   9.32172270e-05])

In [41]:
rf = RandomForestClassifier(n_estimators=50)
rf.fit(x_pca,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [46]:
roc_auc_score(y_train, rf.predict_proba(x_pca)[:,1])

0.9220933338450048

In [47]:
X_normalized_test = preprocessing.normalize(X_test, norm='l2')
x_test_pca = pca.transform(X_normalized_test)

# predicting
y_pred= rf.predict_proba(x_test_pca)[:,1]

submission = pd.DataFrame({"ID":id_test, "TARGET":y_pred})
submission.to_csv("submission_pca.csv", index=False)

print('Completed!')

Completed!


In [ ]:
a= XGBClassifier()

In [10]:
import numpy as np

# 2nd column is redundant
a = np.array([[1, 0, 0], [0, 0, 0], [1, 0, 1]])
print np.linalg.qr(a)[1] # 2nd row empty

# 3rd column is redundant
a = np.array([[1, 0, 0], [1, 0, 1], [0, 0, 0], ])
print np.linalg.qr(a)[1] # 3rd row empty

# No column redundant
a = np.array([[1, 0, 1], 
              [1, 0, 1], 
              [2, 3, 5], ])
print np.linalg.qr(a.T)[1] # No row empty


[[-1.41421356  0.         -0.70710678]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.70710678]]
[[-1.41421356  0.         -0.70710678]
 [ 0.          0.          0.70710678]
 [ 0.          0.          0.        ]]
[[ -1.41421356e+00  -1.41421356e+00  -4.94974747e+00]
 [  0.00000000e+00  -4.74426853e-17   2.12132034e+00]
 [  0.00000000e+00   0.00000000e+00   3.00000000e+00]]


In [9]:
import numpy as np

matrix = np.array(
    [
        [0, 1 ,0 ,0],
        [0, 0, 1, 0],
        [0, 1, 1, 0],
        [1, 0, 0, 1]
    ])

print np.linalg.det(matrix)

for i in range(matrix.shape[0]):
    for j in range(matrix.shape[0]):
        if i != j:
            inner_product = np.inner(
                matrix[:,i],
                matrix[:,j]
            )
            norm_i = np.linalg.norm(matrix[:,i])
            norm_j = np.linalg.norm(matrix[:,j])

            print 'I: ', matrix[:,i]
            print 'J: ', matrix[:,j]
            print 'Prod: ', inner_product
            print 'Norm i: ', norm_i
            print 'Norm j: ', norm_j
            if np.abs(inner_product - norm_j * norm_i) < 1E-5:
                print 'Dependent'
            else:
                print 'Independent'

0.0
I:  [0 0 0 1]
J:  [1 0 1 0]
Prod:  0
Norm i:  1.0
Norm j:  1.41421356237
Independent
I:  [0 0 0 1]
J:  [0 1 1 0]
Prod:  0
Norm i:  1.0
Norm j:  1.41421356237
Independent
I:  [0 0 0 1]
J:  [0 0 0 1]
Prod:  1
Norm i:  1.0
Norm j:  1.0
Dependent
I:  [1 0 1 0]
J:  [0 0 0 1]
Prod:  0
Norm i:  1.41421356237
Norm j:  1.0
Independent
I:  [1 0 1 0]
J:  [0 1 1 0]
Prod:  1
Norm i:  1.41421356237
Norm j:  1.41421356237
Independent
I:  [1 0 1 0]
J:  [0 0 0 1]
Prod:  0
Norm i:  1.41421356237
Norm j:  1.0
Independent
I:  [0 1 1 0]
J:  [0 0 0 1]
Prod:  0
Norm i:  1.41421356237
Norm j:  1.0
Independent
I:  [0 1 1 0]
J:  [1 0 1 0]
Prod:  1
Norm i:  1.41421356237
Norm j:  1.41421356237
Independent
I:  [0 1 1 0]
J:  [0 0 0 1]
Prod:  0
Norm i:  1.41421356237
Norm j:  1.0
Independent
I:  [0 0 0 1]
J:  [0 0 0 1]
Prod:  1
Norm i:  1.0
Norm j:  1.0
Dependent
I:  [0 0 0 1]
J:  [1 0 1 0]
Prod:  0
Norm i:  1.0
Norm j:  1.41421356237
Independent
I:  [0 0 0 1]
J:  [0 1 1 0]
Prod:  0
Norm i:  1.0
Norm j:  1.4142

#Learn Patsy to create independent feats

In [11]:
!pip install patsy

In [5]:
ex_df=df_train[['ID','var3','var15','TARGET']]

In [6]:
ex_df.head()

,ID,var3,var15,TARGET
0,1,2,23,0
1,3,2,34,0
2,4,2,23,0
3,8,2,37,0
4,10,2,39,0


In [7]:
import patsy

In [30]:
f= 'TARGET ~ ID*var3 + ID*var15'
y,X = patsy.dmatrices(f, ex_df, return_type='dataframe')

In [31]:
X.head()

,Intercept,ID,var3,ID:var3,var15,ID:var15
0,1,1,2,2,23,23
1,1,3,2,6,34,102
2,1,4,2,8,23,92
3,1,8,2,16,37,296
4,1,10,2,20,39,390


In [16]:
y.head()

,TARGET
0,0
1,0
2,0
3,0
4,0


In [83]:
# for a numeric dataset
def create_patsy_formula(ex_df, mulfeats=True, addfeats=True, square_feats=True, cubicfeats=True,main_column='TARGET'):
    num_feats = 0 
    def sq(s):
        return s**2
    def cub(s):
        return s**3
    f = main_column+"~"
    for i in range(len(ex_df.columns)-1):
        if square_feats:
            num_feats+=1
            f += "sq("+ex_df.columns[i]+")+"
        if cubicfeats:
            num_feats+=1
            f += "cub("+ex_df.columns[i]+")+"
        for j in range(i+1,len(ex_df.columns)):
            if mulfeats:
                num_feats+=1
                f+= ex_df.columns[i]+"*"+ex_df.columns[j]+"+"
            if addfeats:
                num_feats+=1
                f+= "I("+ex_df.columns[i]+"+"+ex_df.columns[j]+")+"
    return f[:-1],num_feats


In [84]:
f,nums = create_patsy_formula(df_train,addfeats=False,square_feats=False,cubicfeats=False)

In [86]:
X,y = patsy.dmatrices(f, df_train, return_type='dataframe')

RuntimeError: maximum recursion depth exceeded

In [85]:
nums

47278

In [79]:
f

'TARGET~sq(ID)+ID*var3+ID*var15+ID*imp_ent_var16_ult1+ID*imp_op_var39_comer_ult1+ID*imp_op_var39_comer_ult3+ID*imp_op_var40_comer_ult1+ID*imp_op_var40_comer_ult3+ID*imp_op_var40_efect_ult1+ID*imp_op_var40_efect_ult3+ID*imp_op_var40_ult1+ID*imp_op_var41_comer_ult1+ID*imp_op_var41_comer_ult3+ID*imp_op_var41_efect_ult1+ID*imp_op_var41_efect_ult3+ID*imp_op_var41_ult1+ID*imp_op_var39_efect_ult1+ID*imp_op_var39_efect_ult3+ID*imp_op_var39_ult1+ID*imp_sal_var16_ult1+ID*ind_var1_0+ID*ind_var1+ID*ind_var5_0+ID*ind_var5+ID*ind_var6_0+ID*ind_var6+ID*ind_var8_0+ID*ind_var8+ID*ind_var12_0+ID*ind_var12+ID*ind_var13_0+ID*ind_var13_corto_0+ID*ind_var13_corto+ID*ind_var13_largo_0+ID*ind_var13_largo+ID*ind_var13_medio_0+ID*ind_var13+ID*ind_var14_0+ID*ind_var14+ID*ind_var17_0+ID*ind_var17+ID*ind_var18_0+ID*ind_var19+ID*ind_var20_0+ID*ind_var20+ID*ind_var24_0+ID*ind_var24+ID*ind_var25_cte+ID*ind_var26_0+ID*ind_var26_cte+ID*ind_var25_0+ID*ind_var30_0+ID*ind_var30+ID*ind_var31_0+ID*ind_var31+ID*ind_var32_cte

In [69]:
y.columns

Index([u'Intercept', u'sq(ID)', u'ID', u'var3', u'ID:var3', u'I(ID + var3)',
       u'var15', u'ID:var15', u'I(ID + var15)', u'TARGET', u'ID:TARGET',
       u'I(ID + TARGET)', u'sq(var3)', u'var3:var15', u'I(var3 + var15)',
       u'var3:TARGET', u'I(var3 + TARGET)', u'sq(var15)', u'var15:TARGET',
       u'I(var15 + TARGET)'],
      dtype='object')

In [70]:
y.head()

,Intercept,sq(ID),ID,var3,ID:var3,I(ID + var3),var15,ID:var15,I(ID + var15),TARGET,ID:TARGET,I(ID + TARGET),sq(var3),var3:var15,I(var3 + var15),var3:TARGET,I(var3 + TARGET),sq(var15),var15:TARGET,I(var15 + TARGET)
0,1,1,1,2,2,3,23,23,24,0,0,1,4,46,25,0,2,529,0,23
1,1,9,3,2,6,5,34,102,37,0,0,3,4,68,36,0,2,1156,0,34
2,1,16,4,2,8,6,23,92,27,0,0,4,4,46,25,0,2,529,0,23
3,1,64,8,2,16,10,37,296,45,0,0,8,4,74,39,0,2,1369,0,37
4,1,100,10,2,20,12,39,390,49,0,0,10,4,78,41,0,2,1521,0,39
